# Auto cut-off point selection according to the distribution
- When retrieving data based solely on a fixed top_k value, some retrieved items may be semantically irrelevant despite being included in the results. This happens because the function must return top similar - k items regardless of their actual semantic distance. 
- For example, if we set k=5 but only 3 data are truly semantically similar, the function will still include 2 more distant data to meet the k requirement. 
- These outliers can be identified by examining the significant gaps in their distance scores compared to the more relevant results.

Therefore, regardless of the defined `top_k`, a big jump between distances can be used as a `cut-off point`. To this end, this notebook aims to demonstrate 3 different approaches to be used in dynamically selected auto-cut points.

In [18]:
import numpy as np
import plotly.graph_objects as go

from kneed import KneeLocator
from scipy.stats import gaussian_kde

### Method 1: Maximum difference
- Simplest approach
- Calculates the difference between each consecutive pair of distances
- Finds the largest gap/jump in the distances
- Uses the point after this largest gap as the cutoff
- Limitation: Can be sensitive to outliers and might not work well with gradual changes

### Method 2: Elbow Method
-   Treats the distance curve like a person's arm, looking for the "elbow" point
-    Finds where the curve bends most significantly
-   Works well with gradual changes and is less sensitive to outliers
-   Finding the elbow is not possible if the curve is too linear

### Method 3: Density-based analysis

- Uses kernel density estimation to find the point with the lowest density
- Smooths the data using Gaussian kernels to reduce noise
- Using Scott's rule to normalize the data
- Assumes fitting into normal distribution
- Limitation: The amount of smoothing can affect the results

In [ ]:
def analyze_distance_distribution(distances, plot_title="Distance Distribution Analysis"):
    """
    Analyze distance distribution and find potential cutoff points using multiple methods.
    
    Args:
        distances (list): List of distance values in ascending order
        plot_title (str): Title for the plot
    
    Returns:
        dict: Dictionary containing different cutoff points
    """
    # Convert to numpy array and ensure sorted
    distances = np.array(sorted(distances))
    indices = np.arange(len(distances))
    
    # Calculate differences between consecutive points
    differences = np.diff(distances)
    
    # Method 1: Maximum difference
    # Calculates the difference between each consecutive pair of distances
    # Finds the largest gap/jump in the distances
    # Uses the point after this largest gap as the cutoff
    # Limitation: Can be sensitive to outliers and might not work well with gradual changes
    max_diff_idx = np.argmax(differences)
    max_diff_cutoff = distances[max_diff_idx + 1]
    
    # Method 2: Kneedle algorithm aka elbow method
    # Treats the distance curve like a person's arm, looking for the "elbow" point
    # Finds where the curve bends most significantly
    # Works well with gradual changes and is less sensitive to outliers
    # Finding the elbow is not possible if the curve is too linear
    kneedle = KneeLocator(
        indices, 
        distances,
        S=1.0,  # Sensitivity parameter
        curve='convex',
        direction='increasing'
    )
    knee_cutoff = distances[kneedle.knee] if kneedle.knee else None
    
    # Method 3: Density-based analysis
    # Uses kernel density estimation to find the point with the lowest density
    # Smooths the data using Gaussian kernels to reduce noise
    # Limitation: The amount of smoothing can affect the results

    grid = np.linspace(min(distances), max(distances), 100)
    kde = gaussian_kde(distances, bw_method='scott')
    density = kde.evaluate(grid)
    # Exclude edges from consideration
    # This prevents finding cutoffs at the edges where density naturally drops
    exclude_edges = int(len(grid) * 0.1)
    mid_region_idx = slice(exclude_edges, -exclude_edges)

    # Find local minimum in density within the middle region
    local_min_idx = np.argmin(density[mid_region_idx]) + exclude_edges
    density_cutoff = grid[local_min_idx]

    # Find the closest distance value to density_cutoff
    density_cutoff_idx = np.abs(distances - density_cutoff).argmin()

    # Plot 1: Distance Distribution with cutoffs
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=indices, y=distances, mode='lines', name='Distances'))
    fig.add_trace(go.Scatter(x=[max_diff_idx + 1], y=[max_diff_cutoff], mode='markers', 
                             marker=dict(color='red', size=10), name=f'Max Diff Cutoff ({max_diff_cutoff:.3f})'))
    if knee_cutoff:
        fig.add_trace(go.Scatter(x=[kneedle.knee], y=[knee_cutoff], mode='markers', 
                                 marker=dict(color='green', size=10), name=f'Knee Cutoff ({knee_cutoff:.3f})'))
    fig.add_trace(go.Scatter(x=[density_cutoff_idx], y=[distances[density_cutoff_idx]], mode='markers', 
                             marker=dict(color='purple', size=10), name=f'Density Cutoff ({density_cutoff:.3f})'))
    fig.update_layout(title='Distance Distribution with Cutoff Points', xaxis_title='Index', yaxis_title='Distance')
    fig.show()

    # Plot 2: Difference between consecutive points
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=indices[1:], y=differences, mode='lines', name='Differences'))
    fig2.add_trace(go.Scatter(x=[max_diff_idx + 1], y=[differences[max_diff_idx]], mode='markers', 
                              marker=dict(color='red', size=10), name='Maximum Difference Point'))
    fig2.update_layout(title='Differences between Consecutive Points', xaxis_title='Index', yaxis_title='Difference')
    fig2.show()

    # Plot 3: Density distribution

    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=grid, y=density, mode='lines', name='Density'))
    fig3.add_trace(go.Scatter(x=[density_cutoff], y=[density[local_min_idx]], mode='markers', 
                              marker=dict(color='purple', size=10), name=f'Minimum Density Point'))
    fig3.update_layout(title='Kernel Density Estimation', xaxis_title='Distance', yaxis_title='Density')
    fig3.show()
    
    return {
        'max_difference_cutoff': max_diff_cutoff,
        'knee_cutoff': knee_cutoff,
        'density_cutoff': density_cutoff
    }


In [30]:

# Example usage
if __name__ == "__main__":
    # Sample data
    sample_distances = [
        0.150, 0.153, 0.156, 0.162, 0.163, 
        0.164, 0.164, 0.165, 0.167, 0.168, 
        0.169, 0.750, 0.820, 0.890, 0.950
    ]
    
    # Analyze the distribution
    cutoffs = analyze_distance_distribution(sample_distances)
    
    # Print results
    print("\nCutoff Points:")
    print(f"Maximum Difference Method: {cutoffs['max_difference_cutoff']:.3f}")
    print(f"Knee Detection Method: {cutoffs['knee_cutoff']:.3f}" if cutoffs['knee_cutoff'] else "Knee Detection Method: Not found")
    print(f"Density-based Method: {cutoffs['density_cutoff']:.3f}")


Cutoff Points:
Maximum Difference Method: 0.750
Knee Detection Method: 0.169
Density-based Method: 0.586
